In [5]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score    
from sklearn.cross_validation import StratifiedKFold
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [6]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [7]:
# Read data from files
df = pd.read_csv('Reviews.csv')

In [8]:
df.columns = ['Id', 'ProductId', 'UserId', "ProfileName", 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'label', 'Time', 'title', 'abstract']

In [9]:
df[0:2]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,label,Time,title,abstract
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [10]:
df_5 = df.loc[df['label'] == 5]
df_5 = df_5.sample(n=50000, random_state=10)

df_allElse = df.loc[df['label'] != 5]

df = pd.concat([df_allElse, df_5])

### Apply Word2Vec Inversion to get sentence scores

In [11]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprobsentences(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    #prob = prob.groupby("doc").mean()
    return prob

In [12]:
import numpy as np

def getFeatures(probssentences_train, train1):
    features = np.zeros((np.unique(probssentences_train["doc"]).shape[0],11))
    label = []
    for doc in np.unique(probssentences_train["doc"]):
        inx = np.where(probssentences_train["doc"] == doc)[0]
        subset = probssentences_train[0].iloc[inx]
        score = train1['boolean_label'].iloc[doc]
        label.append(score)

        inxFirst = inx[0]
        inxLast = inx[-1]

        features[doc,0] = np.mean(subset) #mean of scores
        features[doc,1] = np.min(subset) #min score
        features[doc,2] = np.max(subset) #max score
        features[doc,3] = np.std(subset) #stdev of scores
        features[doc,4] = probssentences_train[0].iloc[inxFirst] #score of first sentence
        features[doc,5] = probssentences_train[0].iloc[inxLast] #score of last sentences
        features[doc,6] = len(subset) #number of sentences
        features[doc,7] = np.mean(                               #sigmoid (with k = 10)
                            1./(1+np.exp(-10*
                               subset.astype(float)-0.5 ))) 
        features[doc,8] = np.mean(                               #sigmoid (with k = 20)
                            1./(1+np.exp(-20*
                               subset.astype(float)-0.5 )))
        features[doc,9] = np.argmin(subset)-inxFirst #position of min score
        features[doc,10] = np.argmax(subset)-inxFirst #position of max score
    
    return (features, label)

In [13]:
def buildModels(train1, test1, docs, docsTrain, basemodel, label):
    
    #add boolean labels
    train1['boolean_label'] = 1
    for row in train1.iterrows():
        if row[1]['label'] != label:
            train1.set_value(row[0], 'boolean_label', 0)
            
    test1['boolean_label'] = 1
    for row in test1.iterrows():
        if row[1]['label'] != label:
            test1.set_value(row[0], 'boolean_label', 0)
    
    # ****** Split the labeled training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    inxs_pos = np.where(train1["boolean_label"] == 1)[0].tolist()
    inxs_neg = np.where(train1["boolean_label"] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    

    models = [deepcopy(basemodel) for i in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )

    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probssentencesArticle = docprobsentences(docs,models).astype(object)

    probssentences_train = docprobsentences(docsTrain,models).astype(object)
        
    #step 1- get training features
    obj = getFeatures(probssentences_train, train1)
    features = obj[0]
    label = obj[1]

    #step 2- build decision tree
    clf = tree.DecisionTreeClassifier(min_samples_leaf=1000)
    #criterion = 'entropy
    clf = clf.fit(features, label)

    print "decision tree performance on self"
    resultTrain = clf.predict(features)

    print roc_auc_score(label, resultTrain)

    #step 3- build random forest

    forest = RandomForestClassifier()
    forest = forest.fit( features, label )

    print "random forest performance on self"
    resultTrainRF = forest.predict(features)
    print roc_auc_score(label, resultTrainRF)

    #step 4- get test features
    test = getFeatures(probssentencesArticle, test1)
    featuresT = test[0]
    labelsT = test[1]

    #step 5- get test set performance on decision tree 
    result = clf.predict(featuresT)
    scoreDT = roc_auc_score(labelsT, result)

    #step 6- get test set performance on random forest
    clf2 = forest.predict(featuresT)
    scoreRF = roc_auc_score(labelsT, clf2)
    
    return scoreDT, scoreRF, models, probssentencesArticle, probssentences_train, clf, forest

In [16]:
a = datetime.datetime.now().replace(microsecond=0)

y = df['label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=10)

i = 1
avgDT_All = []
avgDT_1 = []
avgDT_2 = []
avgDT_3 = []
avgDT_4 = []
avgDT_5 = []
avgRF_All = []
avgRF_1 = []
avgRF_2 = []
avgRF_3 = []
avgRF_4 = []
avgRF_5 = []

for train_index, test_index in skf:

    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]
    
    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences
    docsTrain = []
    for review in train1["abstract"]:
        result = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences += result
        docsTrain.append(result)

    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 

    print "Parsing sentences from test set"
    # read in the test set as a list of a list of words
    docs = []
    for review in test1["abstract"]:
        docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

    print "Scoring test set"
    tup1 = buildModels(train1, test1, docs, docsTrain, basemodel, 1)
    scoreDT_1 = tup1[0]
    scoreRF_1 = tup1[1]
    print scoreDT_1
    print scoreRF_1
    print "end label 1"
    avgDT_1.append(scoreDT_1)
    avgRF_1.append(scoreRF_1)
    avgDT_All.append(scoreDT_1)
    avgRF_All.append(scoreRF_1)
    models1 = tup1[2]
    ps_test1 = tup1[3]
    ps_train1 = tup1[4]
    clf1 = tup1[5]
    forest1 = tup1[6]
    #(scoreDT, scoreRF, models, probssentencesArticle, probssentences_train, clf, forest)

    tup2 = buildModels(train1, test1, docs, docsTrain, basemodel, 2)
    scoreDT_2 = tup2[0]
    scoreRF_2 = tup2[1]
    print scoreDT_2
    print scoreRF_2
    print "end label 2"
    avgDT_2.append(scoreDT_2)
    avgRF_2.append(scoreRF_2)
    avgDT_All.append(scoreDT_2)
    avgRF_All.append(scoreRF_2)
    models2 = tup2[2]
    ps_test2 = tup2[3]
    ps_train2 = tup2[4]
    clf2 = tup2[5]
    forest2 = tup2[6]
    
    tup3 = buildModels(train1, test1, docs, docsTrain, basemodel, 3)
    scoreDT_3 = tup3[0]
    scoreRF_3 = tup3[1]
    print scoreDT_3
    print scoreRF_3
    print "end label 3"
    avgDT_3.append(scoreDT_3)
    avgRF_3.append(scoreRF_3)
    avgDT_All.append(scoreDT_3)
    avgRF_All.append(scoreRF_3)
    models3 = tup3[2]
    ps_test3 = tup3[3]
    ps_train3 = tup3[4]
    clf3 = tup3[5]
    forest3 = tup3[6]
    
    tup4 = buildModels(train1, test1, docs, docsTrain, basemodel, 4)
    scoreDT_4 = tup4[0]
    scoreRF_4 = tup4[1]
    print scoreDT_4
    print scoreRF_4
    print "end label 4"
    avgDT_4.append(scoreDT_4)
    avgRF_4.append(scoreRF_4)
    avgDT_All.append(scoreDT_4)
    avgRF_All.append(scoreRF_4)
    models4 = tup4[2]
    ps_test4 = tup4[3]
    ps_train4 = tup4[4]
    clf4 = tup4[5]
    forest4 = tup4[6]
    
    tup5 = buildModels(train1, test1, docs, docsTrain, basemodel, 5)
    scoreDT_5 = tup5[0]
    scoreRF_5 = tup5[1]
    print scoreDT_5
    print scoreRF_5
    print "end label 5"
    avgDT_5.append(scoreDT_5)
    avgRF_5.append(scoreRF_5)
    avgDT_All.append(scoreDT_5)
    avgRF_All.append(scoreRF_5)
    models5 = tup5[2]
    ps_test5 = tup5[3]
    ps_train5 = tup5[4]
    clf5 = tup5[5]
    forest5 = tup5[6]

print ""
print "---------------------------------------------------------------------------------"
print ""
print "DECISION TREE"
print "Average for score of 1:"
avg1 = sum(avgDT_1) / float(len(avgDT_1))
print avg1
print "Average for score of 2:"
avg2 = sum(avgDT_2) / float(len(avgDT_2))
print avg2
print "Average for score of 3:"
avg3 = sum(avgDT_3) / float(len(avgDT_3))
print avg3
print "Average for score of 4:"
avg4 = sum(avgDT_4) / float(len(avgDT_4))
print avg4
print "Average for score of 5:"
avg5 = sum(avgDT_5) / float(len(avgDT_5))
print avg5
print ""
print "Average across all models:"
print sum(avgDT_All) / float(len(avgDT_All))

print ""
print "---------------------------------------------------------------------------------"
print ""
print "RANDOM FOREST"
print "Average for score of 1:"
avg1 = sum(avgRF_1) / float(len(avgRF_1))
print avg1
print "Average for score of 2:"
avg2 = sum(avgRF_2) / float(len(avgRF_2))
print avg2
print "Average for score of 3:"
avg3 = sum(avgRF_3) / float(len(avgRF_3))
print avg3
print "Average for score of 4:"
avg4 = sum(avgRF_4) / float(len(avgRF_4))
print avg4
print "Average for score of 5:"
avg5 = sum(avgRF_5) / float(len(avgRF_5))
print avg5
print ""
print "Average across all models:"
print sum(avgRF_All) / float(len(avgRF_All))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: ".." looks like a filename, not markup. You should probably open thi

Building and training w2v models
Parsing sentences from test set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "///" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)


Scoring test set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Parsing sentences from training set
scoring test set
decision tree performance on self
0.876524522935
random forest performance on self
0.985096620444
0.819004851191
0.831607160348
end label 1
Parsing sentences from training set
scoring test set
decision tree performance on self
0.685429415823
random forest performance on self
0.959190723706
0.629907974676
0.706779898
end label 2
Parsing sentences from training set
scoring test set
decision tree performance on self
0.742889268202
random forest performance on self
0.967503017253
0.668564078016
0.724338626442
end label 3
Parsing sentences from training set
scoring test set
decision tree performance on self
0.815228232817
random forest performance on self
0.981749161294
0.719272541714
0.756392260275
end label 4
Parsing sentences from training set
scoring test set
decision tree performance on self
0.823126917253
random forest performance on self
0.9786181749
0.739881150315
0.755520364283
end label 5
('ROUND', 2)
Building and training w2v m

In [ ]:
pickle.dump(ps_test1, open('./pickled_objects/probssentencesFineFoodReviews1.p', 'wb'))
pickle.dump(ps_train1, open('./pickled_objects/probssentencesFineFoodReviews_train1.p', 'wb'))
pickle.dump(ps_test2, open('./pickled_objects/probssentencesFineFoodReviews2.p', 'wb'))
pickle.dump(ps_train2, open('./pickled_objects/probssentencesFineFoodReviews_train2.p', 'wb'))
pickle.dump(ps_test3, open('./pickled_objects/probssentencesFineFoodReviews3.p', 'wb'))
pickle.dump(ps_train3, open('./pickled_objects/probssentencesFineFoodReviews_train3.p', 'wb'))
pickle.dump(ps_test4, open('./pickled_objects/probssentencesFineFoodReviews4.p', 'wb'))
pickle.dump(ps_train4, open('./pickled_objects/probssentencesFineFoodReviews_train4.p', 'wb'))
pickle.dump(ps_test5, open('./pickled_objects/probssentencesFineFoodReviews5.p', 'wb'))
pickle.dump(ps_train5, open('./pickled_objects/probssentencesFineFoodReviews_train5.p', 'wb'))

In [15]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences